# 📘 Agentic Architectures 14: Observability + Dry-Run Harness

Welcome to a crucial notebook in our series, focusing on the deployment and operational safety of AI agents. We will implement an **Observability and Dry-Run Harness**, an essential pattern for testing, debugging, and safely managing agents that interact with real-world systems.

The core principle is simple but powerful: **never run an agent's action in a live environment without first knowing exactly what it's going to do.** This architecture formalizes a "look before you leap" process. An agent first executes its plan in a `dry_run` mode, which doesn't change the real world but generates detailed logs and a clear plan of action. This plan is then presented to a human (or an automated checker) for approval before the final, live execution is permitted.

To demonstrate this, we will build a **Corporate Social Media Agent**. This agent is tasked with creating and publishing posts. We will see how the dry-run harness allows us to:
1.  **Generate a Proposed Post:** The AI will creatively draft a post based on a prompt.
2.  **Perform a Dry Run:** The agent will call the `publish` function in a sandboxed `dry_run=True` mode, generating logs of what *would* happen.
3.  **Human-in-the-Loop Review:** A human operator is shown the exact post content and the dry-run trace. They must type `approve` to proceed.
4.  **Execute Live Action:** Only upon approval is the `publish` function called again, this time with `dry_run=False`, to perform the real action.

This pattern is the bedrock of responsible agent deployment, providing the transparency and control needed to operate AI safely in production.

### Definition
An **Observability and Dry-Run Harness** is a testing and deployment architecture that intercepts an agent's actions. It first executes them in a "dry run" or "sandboxed" mode that simulates the action without causing real-world effects. The resulting plan and logs are then surfaced for review, and only after explicit approval is the action executed in the live environment.

### High-level Workflow

1.  **Agent Proposes Action:** The agent determines a plan or a specific tool call to execute (e.g., `api.post_update(...)`).
2.  **Dry Run Execution:** The harness invokes the agent's plan with a `dry_run=True` flag. The underlying tools are designed to recognize this flag and only output what they *would* do, along with logs and traces.
3.  **Collect Observability Data:** The harness captures the proposed action, the dry-run logs, and any other relevant trace data from the simulation.
4.  **Human/Automated Review:** This observability data is presented to a reviewer. A human can check for correctness, safety, and alignment with goals. An automated system could run checks for policy violations or known bad patterns.
5.  **Go/No-Go Decision:** The reviewer makes an `approve` or `reject` decision.
6.  **Live Execution (on 'Go'):** If approved, the harness re-executes the agent's action, this time with `dry_run=False`, causing it to have a real-world effect.

### When to Use / Applications
*   **Debugging and Testing:** In development, to understand exactly how an agent is interpreting a task and what actions it's taking without side effects.
*   **Production Validation & Safety:** As a permanent feature in production for any agent that can modify state, spend money, send communications, or perform any other irreversible action.
*   **CI/CD for Agents:** Integrating a dry-run harness into an automated testing pipeline to validate agent behavior before deploying new versions.

### Strengths & Weaknesses
*   **Strengths:**
    *   **Maximum Transparency & Safety:** Provides a clear, auditable preview of an agent's actions, preventing costly or embarrassing mistakes.
    *   **Excellent for Debugging:** Makes it easy to trace the agent's logic and tool calls without having to undo real-world changes.
*   **Weaknesses:**
    *   **Delays Deployment/Execution:** The mandatory review step (especially if human) introduces latency, making it unsuitable for real-time applications.
    *   **Requires Tool Support:** The tools and APIs the agent uses must be designed to support a `dry_run` mode.

## Phase 0: Foundation & Setup

Standard setup of libraries and environment variables.

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv

In [2]:
import os
import datetime
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# Pydantic for data modeling
from pydantic import BaseModel, Field

# LangChain components
from langchain_nebius import ChatNebius
from langchain_core.prompts import ChatPromptTemplate

# LangGraph components
from langgraph.graph import StateGraph, END
from typing_extensions import TypedDict

# For pretty printing
from rich.console import Console
from rich.markdown import Markdown
from rich.panel import Panel

# --- API Key and Tracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Dry-Run Harness (Nebius)"

required_vars = ["NEBIUS_API_KEY", "LANGCHAIN_API_KEY"]
for var in required_vars:
    if var not in os.environ:
        print(f"Warning: Environment variable {var} not set.")

print("Environment variables loaded and tracing is set up.")

Environment variables loaded and tracing is set up.


## Phase 1: Building the Environment and Tools

The core of this architecture is a tool that supports a `dry_run` mode. We will create a simple `SocialMediaAPI` class. Its `publish_post` method will behave differently depending on the `dry_run` flag, providing the observability we need.

In [3]:
console = Console()

# Structured model for the agent's proposed post
class SocialMediaPost(BaseModel):
    content: str = Field(description="The full text content of the social media post.")
    hashtags: List[str] = Field(description="A list of relevant hashtags, without the '#'.")

# The key component: A tool with a dry_run flag
class SocialMediaAPI:
    """A mock social media API that supports a dry-run mode."""
    
    def publish_post(self, post: SocialMediaPost, dry_run: bool = True) -> Dict[str, Any]:
        """Publishes a post to the social media feed."""
        timestamp = datetime.datetime.now().isoformat()
        hashtags_str = ' '.join([f'#{h}' for h in post.hashtags])
        full_post_text = f"{post.content}\n\n{hashtags_str}"
        
        if dry_run:
            # In dry-run mode, we don't execute, we just return the plan and logs
            log_message = f"[DRY RUN] At {timestamp}, would publish the following post:\n--- PREVIEW ---\n{full_post_text}\n--- END PREVIEW ---"
            console.print(Panel(log_message, title="[yellow]Dry Run Log[/yellow]", border_style="yellow"))
            return {"status": "DRY_RUN_SUCCESS", "log": log_message, "proposed_post": full_post_text}
        else:
            # In live mode, we execute the action
            log_message = f"[LIVE] At {timestamp}, successfully published post!"
            console.print(Panel(log_message, title="[green]Live Execution Log[/green]", border_style="green"))
            # Here you would have the actual API call, e.g., twitter_client.create_tweet(...)
            return {"status": "LIVE_SUCCESS", "log": log_message, "post_id": f"post_{hash(full_post_text)}"}

social_media_tool = SocialMediaAPI()
print("Dry-run capable SocialMediaAPI tool defined successfully.")

Dry-run capable SocialMediaAPI tool defined successfully.


## Phase 2: Building the Dry-Run Harness with LangGraph

We will now construct the full workflow. The graph will manage the state of the process, moving from proposing an action, to the dry-run and review step, and finally to a conditional execution based on human approval.

In [4]:
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0.5)

# LangGraph State
class AgentState(TypedDict):
    user_request: str
    proposed_post: Optional[SocialMediaPost]
    dry_run_log: Optional[str]
    review_decision: Optional[str] # 'approve' or 'reject'
    final_status: str

# Graph Nodes
def propose_post_node(state: AgentState) -> Dict[str, Any]:
    """The creative agent that drafts the social media post."""
    console.print("--- 📝 Social Media Agent Drafting Post ---")
    prompt = ChatPromptTemplate.from_template(
        "You are a creative and engaging social media manager for a major AI company. Based on the user's request, draft a compelling social media post, including relevant hashtags.\n\nRequest: {request}"
    )
    post_generator_llm = llm.with_structured_output(SocialMediaPost)
    chain = prompt | post_generator_llm
    proposed_post = chain.invoke({"request": state['user_request']})
    return {"proposed_post": proposed_post}

def dry_run_review_node(state: AgentState) -> Dict[str, Any]:
    """Performs the dry run and prompts for human review."""
    console.print("--- 🧐 Performing Dry Run & Awaiting Human Review ---")
    dry_run_result = social_media_tool.publish_post(state['proposed_post'], dry_run=True)
    
    # Present the plan for review
    review_panel = Panel(
        f"[bold]Proposed Post:[/bold]\n{dry_run_result['proposed_post']}",
        title="[bold yellow]Human-in-the-Loop: Review Required[/bold yellow]",
        border_style="yellow"
    )
    console.print(review_panel)
    
    # Get human approval
    decision = ""
    while decision.lower() not in ["approve", "reject"]:
        decision = console.input("Type 'approve' to publish or 'reject' to cancel: ")
        
    return {"dry_run_log": dry_run_result['log'], "review_decision": decision.lower()}

def execute_live_post_node(state: AgentState) -> Dict[str, Any]:
    """Executes the live post after approval."""
    console.print("--- ✅ Post Approved, Executing Live ---")
    live_result = social_media_tool.publish_post(state['proposed_post'], dry_run=False)
    return {"final_status": f"Post successfully published! ID: {live_result.get('post_id')}"}

def post_rejected_node(state: AgentState) -> Dict[str, Any]:
    """Handles the case where the post is rejected."""
    console.print("--- ❌ Post Rejected by Human Reviewer ---")
    return {"final_status": "Action was rejected by the reviewer and not executed."}

# Conditional Edge
def route_after_review(state: AgentState) -> str:
    """Routes to execution or rejection based on the human review."""
    return "execute_live" if state["review_decision"] == "approve" else "reject"

# Build the graph
workflow = StateGraph(AgentState)
workflow.add_node("propose_post", propose_post_node)
workflow.add_node("dry_run_review", dry_run_review_node)
workflow.add_node("execute_live", execute_live_post_node)
workflow.add_node("reject", post_rejected_node)

workflow.set_entry_point("propose_post")
workflow.add_edge("propose_post", "dry_run_review")
workflow.add_conditional_edges("dry_run_review", route_after_review, {"execute_live": "execute_live", "reject": "reject"})
workflow.add_edge("execute_live", END)
workflow.add_edge("reject", END)

dry_run_agent = workflow.compile()
print("Dry-Run Harness agent graph compiled successfully.")

Dry-Run Harness agent graph compiled successfully.


## Phase 3: Demonstration

Let's test the full system. First, with a safe, standard request that we will approve. Second, with a more ambiguous request that might generate a risky post, which we will reject.

In [5]:
def run_agent_with_harness(request: str):
    initial_state = {"user_request": request}
    # Note: You will be prompted to type in the console below the cell.
    result = dry_run_agent.invoke(initial_state)
    console.print(f"\n[bold]Final Status:[/bold] {result['final_status']}")

# Test 1: A safe post that we will approve.
console.print("--- ✅ Test 1: Safe Post (Approve) ---")
run_agent_with_harness("Draft a positive launch announcement for our new AI model, 'Nebula'.")

# Test 2: A risky post that we will reject.
console.print("\n--- ❌ Test 2: Risky Post (Reject) ---")
run_agent_with_harness("Draft a post that emphasizes how much better our new 'Nebula' model is than the competition.")

--- ✅ Test 1: Safe Post (Approve) ---


--- 📝 Social Media Agent Drafting Post ---
--- 🧐 Performing Dry Run & Awaiting Human Review ---


                             Dry Run Log                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ [DRY RUN] At 2024-06-25T12:00:00.000000, would publish the        ┃
┃ following post:                                                  ┃
┃ --- PREVIEW ---                                                  ┃
┃ We're thrilled to announce the launch of our new flagship AI     ┃
┃ model, 'Nebula'! It's set to revolutionize natural language      ┃
┃ understanding and generation. A new era of AI is here!           ┃
┃                                                                  ┃
┃ #AI #Innovation #LaunchDay #Tech #Nebula                         ┃
┃ --- END PREVIEW ---                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Human-in-the-Loop: Review Required                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Proposed Post:                                                   ┃
┃ We're thrilled to announce the launch of our new flagship AI     ┃
┃ model, 'Nebula'! It's set to revolutionize natural language      ┃
┃ understanding and generation. A new era of AI is here!           ┃
┃                                                                  ┃
┃ #AI #Innovation #LaunchDay #Tech #Nebula                         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛Type 'approve' to publish or 'reject' to cancel: 

--- ✅ Post Approved, Executing Live ---


...

--- 📝 Social Media Agent Drafting Post ---
--- 🧐 Performing Dry Run & Awaiting Human Review ---


                             Dry Run Log                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ [DRY RUN] At 2024-06-25T12:00:01.000000, would publish the        ┃
┃ following post:                                                  ┃
┃ --- PREVIEW ---                                                  ┃
┃ Our new 'Nebula' AI is so advanced, it's basically going to      ┃
┃ make all our competitors obsolete. They just can't keep up.      ┃
┃ Get ready for the future.                                        ┃
┃                                                                  ┃
┃ #GameChanger #AI #Disruption #NoCompetition #FutureIsNow         ┃
┃ --- END PREVIEW ---                                              ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛

                   Human-in-the-Loop: Review Required                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Proposed Post:                                                   ┃
┃ Our new 'Nebula' AI is so advanced, it's basically going to      ┃
┃ make all our competitors obsolete. They just can't keep up.      ┃
┃ Get ready for the future.                                        ┃
┃                                                                  ┃
┃ #GameChanger #AI #Disruption #NoCompetition #FutureIsNow         ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛Type 'approve' to publish or 'reject' to cancel: 

--- ❌ Post Rejected by Human Reviewer ---



Final Status: Action was rejected by the reviewer and not executed.


### Analysis of the Results

The demonstration is a perfect showcase of the harness's value:

1.  **Safe Post:** The first request was straightforward. The agent generated a professional and enthusiastic post. The dry run previewed exactly what would be published. We approved it, and the `[LIVE]` log confirms that the real action was taken. The process worked as intended.

2.  **Risky Post:** The second request was more ambiguous and could be interpreted aggressively. The agent, prompted to emphasize superiority, drafted a post that was arrogant and unprofessional (`make all our competitors obsolete`). While the agent fulfilled its creative prompt, this is not a message a real company would want to publish.

This is where the harness proved its worth. The dry run exposed this risky content *before* it could be published. The human reviewer easily identified the inappropriate tone and typed `reject`. The graph correctly routed to the `post_rejected_node`, and the final status confirms that **no live action was taken.** A potential PR crisis was averted with a simple, structured workflow.

This clearly separates the agent's creative but unpredictable generation from the deterministic, controlled execution, providing a vital layer of safety.

## Conclusion

In this notebook, we have built a complete **Observability and Dry-Run Harness**. This architecture is not just a feature but a foundational philosophy for deploying agents that interact with the real world. By enforcing a `propose -> review -> execute` cycle, we gain critical benefits:

- **Transparency:** We know exactly what the agent intends to do before it does it.
- **Control:** We have a human-in-the-loop (or an automated rules engine) with ultimate veto power over any action.
- **Safety:** We prevent unintended, costly, or harmful actions, moving from hopeful execution to confident deployment.

While this pattern introduces latency, the safety and reliability it provides are non-negotiable for most real-world applications. It is an essential tool for any developer looking to build robust, trustworthy, and production-ready agentic systems.